In [1]:
#general

import pandas as pd
import numpy as np
import os
import pickle
import time
import sys


import tensorflow as tf

#for utility packages
from Utilities.utilities import import_data
from Utilities.utilities import reset_graph

from functools import partial

from tan.tan_util import get_tan_nll as tan
from tan.tan_util import get_tan_nll_cond as tan_cond

from tensorflow.contrib.slim.nets import resnet_v2
import imagenet_helper_files.vgg_preprocessing
import tensorflow.contrib.slim as slim

#For parsing records once written
from Utilities.set_record_parser import build_set_dataset
from Utilities.set_record_parser import get_file_lists
from Utilities.models import log_dir_build
import pickle

c:\users\sdgeo\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Setup Directories 

Here we are going to get the files needed to do the project

In [2]:
#Locate Neccesary Files
homogenous_train = open("D:/Machine_Learning/Datasets/cortical_images/L/Original_Images_Train.pkl", "rb")
nonhomogenous_train = open("D:/Machine_Learning/Datasets/cortical_images/L/Inhomogeneous_Images_Train.pkl", "rb")
homogenous_test = open("D:/Machine_Learning/Datasets/cortical_images/L/Original_Images_Test.pkl", "rb")
nonhomogenous_test = open("D:/Machine_Learning/Datasets/cortical_images/L/Inhomogeneous_Images_Test.pkl", "rb")
train_homog = pickle.load(homogenous_train)
train_homog_labels = np.zeros(train_homog.shape[0])
train_nonhomog = pickle.load(nonhomogenous_train)
train_nonhomog_labels = np.ones(train_homog.shape[0])
test_homog = pickle.load(homogenous_test)
test_homog_labels = np.zeros(test_homog.shape[0])
test_nonhomog = pickle.load(nonhomogenous_test)
test_nonhomog_labels = np.ones(test_homog.shape[0])
homogenous_train.close()
nonhomogenous_train.close()
homogenous_test.close()
nonhomogenous_test.close()

In [3]:
#combine into full train and test set as well as ground truth
full_train = np.expand_dims(np.concatenate((train_homog, train_nonhomog), axis = 0), axis=4)
full_train_labels = np.concatenate((train_homog_labels, train_nonhomog_labels), axis=0)

full_test = np.expand_dims(np.concatenate((test_homog, test_nonhomog), axis = 0), axis=4)
full_test_labels = np.concatenate((test_homog_labels, test_nonhomog_labels), axis=0)

In [4]:
image_1 = full_test[0]
full_test_labels.shape

(160,)

# Design Neural Network

The following provides the code to import and use the TF_Records for the set project

In [20]:
#directory for logs in training
set_net_logs = 'D:/AI/models/set_project_brain/logs'
model_path = log_dir_build(set_net_logs, "set_project_brain")
#model_path = 'D:/AI/models/set_project_brain/logs/set_project-run-20190306174848/'

#directory for all the models saved during training
set_net_model = 'D:/AI/models/set_project_brain/model/' + 'set_project'
set_net_model_best = 'D:/AI/models/set_project_brain/model/' + 'set_project_best'
res_net_model = "D:/AI/models/res_net/v2_50/resnet_v2_50.ckpt"

In [21]:
def deep_set_layer(input_code, input_size, condense_size, set_size=3, layer_name='default', activation_func=tf.nn.sigmoid):
    
    learned_transform = tf.get_variable(layer_name + '_transform', shape=[input_size,condense_size], 
                                        trainable=True, initializer=tf.contrib.layers.variance_scaling_initializer()) 
    batched_transform = tf.broadcast_to(learned_transform, [tf.shape(input_code)[0], input_size, condense_size])
    transform_layer = tf.matmul(input_code, batched_transform)
    activation = activation_func(transform_layer)
    
    
    lambda_1 = tf.get_variable(layer_name + "_lambda", [condense_size], trainable=True, dtype=tf.float32, initializer=tf.initializers.random_normal(mean=1))
    lambda_1_transformed = tf.broadcast_to(lambda_1, [set_size, condense_size])
    multipy_pairwise = tf.broadcast_to(lambda_1_transformed, [tf.shape(input_code)[0], set_size, condense_size])
    
    sigma_1 = tf.abs(tf.get_variable(layer_name + "_sigma", [condense_size], trainable=True, dtype=tf.float32, initializer=tf.initializers.random_normal(mean=0)))
    sigma_1_tranformed = tf.broadcast_to(sigma_1, [tf.shape(input_code)[0], condense_size])
    

    # + sigma * mean(Data)
    max_pool_1 = tf.reduce_mean(activation, axis=1)
    sum_term = tf.multiply(sigma_1_tranformed, max_pool_1)
    sum_term_final = tf.expand_dims(sum_term, axis=1)  
    
    pre_activation_1 = tf.multiply(activation, multipy_pairwise) + sum_term_final
    layer_1 = activation_func(pre_activation_1)
    return layer_1

def pad_and_input(original_image):
    new_image = tf.image.resize_image_with_crop_or_pad(original_image, 331,331)
    new_image = tf.image.grayscale_to_rgb(new_image)
    return new_image

def resnet_build(input):
    input = pad_and_input(input)
    net, end_points = resnet_v2.resnet_v2_50(input, is_training=False)
    saver = tf.train.Saver(name="Original_Saver")
    return(net, end_points, saver)

def get_resnet_estimate(item):
    item = pad_and_input(item)
    net, end_points = resnet_v2.resnet_v2_50(item, reuse=True,is_training=False)
    net = tf.layers.flatten(net)
    return(net)

In [22]:
reset_graph()

#Placeholder for choosing input, epochs, batches, and datasets at runtime
learning_rate_class = .01
dropout_rate = 0.2
set_size = 3
    
with tf.name_scope('Data_Retrieval'):
    #put the data in the graph
    batch_size = tf.placeholder_with_default(30, shape=[], name= "Batch_Size")
    training = tf.placeholder_with_default(True, shape=(), name = 'training')
    data_set = tf.placeholder(shape=[None,set_size,128,128,1], name="All_Data", dtype=tf.float32)
    image_set = tf.placeholder(shape=[None,set_size,331,331,3], name="All_Data", dtype=tf.float32)
    data_label = tf.placeholder(shape=[None], name="Data_Labels", dtype=tf.int32)  
    data_sample = tf.placeholder(shape=[set_size,128,128,1], name="All_Data_sample", dtype=tf.float32)
    image_sample = tf.placeholder(shape=[set_size,331,331,3], name="All_Data_sample", dtype=tf.float32)


with tf.name_scope("BN_Layer_AE_Layers"):
    #Define initalizer and batch normalization layers
    bn_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)
    he_init = tf.contrib.layers.variance_scaling_initializer()

with slim.arg_scope(resnet_v2.resnet_arg_scope()):
    final_net_output, end_point, saver_resnet = resnet_build(data_sample)
    encoding_raw = tf.map_fn(lambda x: get_resnet_estimate(x), data_set, dtype=(tf.float32))
    encoding = tf.stop_gradient(encoding_raw)
print(encoding)

with tf.name_scope("Set_Analyzer"):
    #the network for generating output of our set
    with tf.name_scope('Unique_Identify'):
        code_size = 2048
        n_layer_1 = 1000
        n_layer_2 = 500
        n_layer_3 = 250
        n_unq_1 = 100
        n_unq_2 = 10
        n_unq_final = 2
        deep_activation = tf.nn.relu
        
        batch_item = batch_size
        
        with tf.name_scope('DeepSet_Layer_1'):
            #1000
            deep_1 = deep_set_layer(encoding, code_size, n_layer_1, set_size=3, layer_name='Deep_One', activation_func=deep_activation)
        
        with tf.name_scope('DeepSet_Unq_Layer_1'):
            #500 Output
            deep_unq_1 = deep_set_layer(deep_1, n_layer_1, n_layer_2, set_size=3, layer_name='Deep_Unq_One', activation_func=deep_activation)
            
        with tf.name_scope('DeepSet_Unq_Layer_2'):
            #250 Output
            deep_unq_2 = deep_set_layer(deep_unq_1, n_layer_2, n_layer_3, set_size=3, layer_name='Deep_Unq_Two', activation_func=deep_activation)
            
        with tf.name_scope('Final_Unq_Deep_Pool'):
            #250 Output
            final_unq_deep_layer = tf.reduce_sum(deep_unq_2, 1)
            
        with tf.name_scope("Unq_Hidden_Layer_1"):
            #100 Output
            hidden1_unq = tf.layers.dense(final_unq_deep_layer, n_unq_1, name="hidden1_unq", kernel_initializer=he_init)
            hidden1_drop_unq = tf.layers.dropout(hidden1_unq, dropout_rate, training=training)
            hidden1_cast_unq = tf.cast(hidden1_drop_unq, tf.float32)
            bn1_cat_unq = bn_batch_norm_layer(hidden1_cast_unq)
            bn1_act_cat_unq = tf.nn.relu(bn1_cat_unq)  
            
        with tf.name_scope("Unq_Hidden_Layer_2"):
            #10 Output
            hidden2_unq = tf.layers.dense(bn1_act_cat_unq, n_unq_2, name="hidden2_unq", kernel_initializer=he_init)
            hidden2_drop_unq = tf.layers.dropout(hidden2_unq, dropout_rate, training=training)
            bn2_cat_unq = bn_batch_norm_layer(hidden2_drop_unq)
            bn2_act_cat_unq = tf.nn.relu(bn2_cat_unq)  
        
        
        with tf.name_scope("Final_Layer_unq"): 
            #Get softmax
            logits_before_bn_unq = tf.layers.dense(bn2_act_cat_unq, n_unq_final, name="outputs_unq")
            logits_unq = bn_batch_norm_layer(logits_before_bn_unq, name="logits_unq")
            softmax_unq = tf.nn.softmax(logits_unq, name="final_soft_max_unq")
            
            
        with tf.name_scope("Unique_loss"):           
            
            #Get cross entropy from labels
            xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=data_label, logits=logits_unq)
            loss_unq = tf.reduce_mean(xentropy, name="loss_unq")
            loss_summary_unq = tf.summary.scalar('loss_summary_unq', loss_unq)
            
        with tf.name_scope("eval_unq"):
            correct = tf.nn.in_top_k(logits_unq, data_label, 1)
            accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
            accuracy_summary = tf.summary.scalar('accuracy_summary', accuracy)
                
        with tf.name_scope("unique_train"):
            global_step_unique = tf.Variable(0, trainable=False, name='global_step_unique')
            optimizer_unq = tf.train.AdamOptimizer(learning_rate=learning_rate_class)

            extra_update_ops_unq = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

            with tf.control_dependencies(extra_update_ops_unq):
                 training_op_unq = optimizer_unq.minimize(loss_unq, global_step=global_step_unique)

init = tf.global_variables_initializer()    
saver_total = tf.train.Saver(name="Full_Graph")

Tensor("StopGradient:0", shape=(?, 3, 2048), dtype=float32)


In [23]:
#Write Graph to log directory
filewriter = tf.summary.FileWriter(model_path, tf.get_default_graph())

In [24]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    variable_parameters = 1
    for dim in shape:
        variable_parameters *= dim.value
    total_parameters += variable_parameters
print(total_parameters)

26222216


In [25]:
#Initialize the network

with tf.Session() as sess:
    init.run()
    saver_resnet.restore(sess,res_net_model)
    saver_total.save(sess, set_net_model)
    saver_total.save(sess, set_net_model_best)
   

INFO:tensorflow:Restoring parameters from D:/AI/models/res_net/v2_50/resnet_v2_50.ckpt


In [26]:
print(model_path)

D:/AI/models/set_project_brain/logs/set_project_brain-run-20190421200123/


# Train the Network

Train the network to both generate conditioning for the network and also classify the type of set

In [27]:
epochs = 1000
steps_between_test_save = 1
batch = 15
train_size = 600
#all_data_steps = np.int(np.floor(train_size/batch))
all_data_steps = 2
worst_acc = 0


with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    #restore saver, build iterator, set the step to the global step
    saver_total.restore(sess, set_net_model)
    
    #Set up the global steps
    step = 1
    print("Loaded model. Training network initially. Logs into: " + model_path)
    
    #Iterate through training 
    while step < epochs:
        for i in range(all_data_steps):
            idx = np.random.randint(full_train.shape[0], size=batch)
            batch_train = full_train[idx,:]
            batch_train_label = full_train_labels[idx]
            
            #run Training Op
            sess.run([training_op_unq], feed_dict={data_set: batch_train, data_label: batch_train_label})
        
        #see if we are improving on the test data
        #Maybe Test Accuracy
        if ((step % steps_between_test_save) == 0) :
            idx = np.random.randint(full_test.shape[0], size=batch)
            batch_test = full_test[idx,:]
            batch_test_label = full_test_labels[idx]
            
            loss_un, loss_un_val, acc, acc_sum = sess.run([loss_summary_unq, loss_unq, accuracy, accuracy_summary], 
                                                   feed_dict = {data_set: batch_test , data_label: batch_test_label, training: False, batch_size: batch})

            filewriter.add_summary(loss_un, step)
            filewriter.add_summary(acc_sum, step)
            print("Epoch: " + str(step) + " Unique Loss: " + str(loss_un_val) + " Accuracy: " + str(acc))
            if acc > worst_acc:
                saver_total.save(sess, set_net_model_best)
                worst_acc = acc
            saver_total.save(sess, set_net_model)
        step = step + 1
            
    #Finish the final Model
    saver_total.save(sess, set_net_model)
    end_time = time.time()
    total_steps = tf.train.global_step(sess, global_step_unique)
    final_time = end_time - start_time
    print("Did " + str(total_steps) + " of loss minimized training in " + str(final_time) + " seconds.")

INFO:tensorflow:Restoring parameters from D:/AI/models/set_project_brain/model/set_project
Loaded model. Training network initially. Logs into: D:/AI/models/set_project_brain/logs/set_project_brain-run-20190421200123/
Epoch: 1 Unique Loss: 8.830122 Accuracy: 0.33333334
Epoch: 2 Unique Loss: 48.345074 Accuracy: 0.53333336
Epoch: 3 Unique Loss: 4.2135034 Accuracy: 0.6666667
Epoch: 4 Unique Loss: 4.4252152 Accuracy: 0.53333336
Epoch: 5 Unique Loss: 2.7327025 Accuracy: 0.73333335
Epoch: 6 Unique Loss: 1.367785 Accuracy: 0.73333335
Epoch: 7 Unique Loss: 1.5365306 Accuracy: 0.6
Epoch: 8 Unique Loss: 1.805729 Accuracy: 0.33333334
Epoch: 9 Unique Loss: 1.1966574 Accuracy: 0.46666667
Epoch: 10 Unique Loss: 1.1794622 Accuracy: 0.33333334
Epoch: 11 Unique Loss: 0.63341635 Accuracy: 0.6666667
Epoch: 12 Unique Loss: 0.74161476 Accuracy: 0.46666667
Epoch: 13 Unique Loss: 0.7159379 Accuracy: 0.53333336
Epoch: 14 Unique Loss: 0.6882683 Accuracy: 0.6
Epoch: 15 Unique Loss: 0.78807575 Accuracy: 0.466666

KeyboardInterrupt: 

# Test the Training

Here we will see how the network performs after training

In [ ]:
batch = 30
epochs = 100
num_iter = 200
with tf.Session() as sess:
    saver_total.restore(sess, set_net_model)
    #initialize iterator
    sess.run(val_iterator.initializer, feed_dict={filename: val_list, batch_size: batch, num_epochs:epochs})
    val_handle = sess.run(val_iterator.string_handle())
    
    right, guess, acc, cond, ims, cc = sess.run([correct_class, classes_guess, class_accuracy, conditionals, file_data, code_data], feed_dict={handle: val_handle, training: False})

In [ ]:
#Get final parameter count
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    variable_parameters = 1
    for dim in shape:
        variable_parameters *= dim.value
    total_parameters += variable_parameters
print(total_parameters)

In [ ]:
item_number = 3

In [ ]:
ims[item_number]

In [ ]:
core_code = cc[item_number]
for i in core_code:
    from matplotlib import pyplot as plt
    plt.plot(i)
    plt.axis('off')
    plt.show()

In [ ]:
in_set = np.where(right[item_number] == 1)[0]
for i in in_set:
    print('Array_Position: ' + str(i)  + ' Class_Label: ' + label_file[i])
right[item_number]

In [ ]:
in_set = np.where(guess[item_number] == 1)[0]
for i in in_set:
    print('Array_Position: ' + str(i)  + ' Class_Label: ' + label_file[i])
guess[item_number]

In [ ]:
acc

In [ ]:
cond[item_number][2]

# Final Training Metrics

Here we will get final accuracies for both tasks

In [ ]:
epochs = 2000
steps_between_test_save = 1
batch = 30
test_size = 26000
all_data_steps = np.int(np.floor(test_size/batch))
unique_accuracy = 0
identify_accuracy = 0

with tf.Session() as sess:
    start_time = time.time()
    #restore saver, build iterator, set the step to the global step
    saver_total.restore(sess, set_net_model)
    
    print("Loaded model. Training network initially. Logs into: " + model_path)
    
    #initialize iterator
    sess.run(val_iterator.initializer, feed_dict={filename: val_list, batch_size: batch, num_epochs:epochs})
    val_handle = sess.run(val_iterator.string_handle())
    
    for i in range(all_data_steps):
        unq_acc, ide_acc = sess.run([accuracy, class_accuracy], feed_dict = {handle: val_handle, training: False})
        unique_accuracy = ((i  * unique_accuracy) + unq_acc)/(i + 1)
        identify_accuracy = ((i  * identify_accuracy) + ide_acc)/(i + 1)
    

In [ ]:
unique_accuracy

In [ ]:
identify_accuracy

# Network Demonstration

Here we are going to show running the bones of the network and look a the training data 

In [ ]:
with tf.Session() as sess:
    #Initialize Train and validation iterator
    sess.run(train_iterator.initializer, feed_dict={filename: train_list, batch_size: 1, num_epochs:1})
    sess.run(val_iterator.initializer, feed_dict={filename: val_list, batch_size: 1, num_epochs:1})
    train_handle = sess.run(train_iterator.string_handle())
    val_handle = sess.run(val_iterator.string_handle())

    test_code, test_class, test_file, test_unique = sess.run([code, class_data, file_data, uniques], feed_dict={handle: val_handle,training: False})

In [ ]:
#Show codes from single example
for i in range(0,3):
    X_test = test_code[0][i]
    x_val = X_test

    from matplotlib import pyplot as plt
    plt.plot(x_val)
    plt.axis('off')
    plt.show()

In [ ]:
#Show class for each item in set
for i in range(0,3):
    X_test = test_class[0][i]
    print('Class: ' + str(X_test) + ' Label: ' + labels[X_test])

In [ ]:
#Show what images the codes came from
print(test_file)

In [ ]:
#Show how many unitque Items are in the set
test_unique[0]